In [1]:
'''# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.'''

'# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load in \n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the "../input/" directory.\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# Any results you write to the current directory are saved as output.'

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential
%matplotlib inline

In [3]:
train_dir = './chest_xray/train'
test_dir = './chest_xray/test'
val_dir = './chest_xray/val'


Data Augmentation : 
    Changing the shape and size of data to create more examples

In [4]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
batch_size = 16
img_width, img_height = 150,150
input_shape = (img_width,img_height,3)
epochs = 20

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [8]:
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size = (img_width,img_height),
    batch_size = batch_size,
    class_mode = 'binary')

Found 16 images belonging to 2 classes.


In [9]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_width,img_height),
    batch_size = batch_size,
    class_mode = 'binary')

Found 624 images belonging to 2 classes.


Define CNN Model

In [10]:
model = Sequential([
    Conv2D(32,(3,3), input_shape=input_shape,activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(64, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation = 'sigmoid')
])

In [11]:
model.compile(loss='binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [12]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 326,
    epochs = epochs,
    validation_data = val_generator,
    validation_steps = 1)

Epoch 1/20
326/326 [==============================] - 117s 358ms/step - loss: 0.3962 - accuracy: 0.8156 - val_loss: 0.4865 - val_accuracy: 0.8750
Epoch 2/20
326/326 [==============================] - 98s 302ms/step - loss: 0.2741 - accuracy: 0.8785 - val_loss: 1.2091 - val_accuracy: 0.6250
Epoch 3/20
326/326 [==============================] - 101s 309ms/step - loss: 0.2524 - accuracy: 0.8974 - val_loss: 0.8061 - val_accuracy: 0.8125
Epoch 4/20
326/326 [==============================] - 97s 298ms/step - loss: 0.2381 - accuracy: 0.9076 - val_loss: 0.8244 - val_accuracy: 0.7500
Epoch 5/20
326/326 [==============================] - 99s 305ms/step - loss: 0.2141 - accuracy: 0.9155 - val_loss: 0.8436 - val_accuracy: 0.6250
Epoch 6/20
326/326 [==============================] - 96s 295ms/step - loss: 0.2037 - accuracy: 0.9220 - val_loss: 0.7564 - val_accuracy: 0.6250
Epoch 7/20
326/326 [==============================] - 98s 302ms/step - loss: 0.1867 - accuracy: 0.9252 - val_loss: 0.5988 - val_

In [13]:
scores = model.evaluate_generator(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*180))


accuracy: 156.63%
